In [2]:
#############################################Ethan L. Mines##############################
import numpy as np
import cntk as C



In [ ]:
dataPath = "."
dataFile = "arizona.csv"
data = np.genfromtext(dataPath + dataFile, delimiter = ",", dtype = str)

In [6]:
numType = np.float32
numLayers = 1

num_training = 30000

In [2]:
#Expects number of possible categories "num_categories" and category index "category" in range 0...num_categories-1
def toOneHot(category, num_categories):
    oneHot = np.zeros(num_categories, dtype = numType)
    oneHot[category] = 1
    return oneHot

def weekdayIndex(weekday):
    return int(weekday) - 1
def monthIndex(month):
    return int(month) - 1
def zoneIndex(zone, zoneDict):
    return zipDict[zoneCode]
def crimeIndex(crime, crimeDict):
    return crimeDict[crime]

#############Returns str->int dictionary where the int's are indices for one-hot vectors of categories in column
def extractCategories(column):
    dictionary = {}
    i = 0
    for element in column:
        if element not in dictionary:
            dictionary[element] = i
    return dictionary


    

#Days and period are static across campuses, but the zones could depend on how many zones Tam defines for each campus 
zoneIndices = extractCategories()



        


#Length-7 one-hot encoded vectors for days of week
#Length-3 one-hot encoded vectors for period of day

#OUtput-softmax probabilities for Tam's designated zones

In [4]:
num_features = 4 #Month, day of week, hour
num_zone_classes = 5#len(zoneIndices)
num_crime_classes = 5#len(crimeIndices)

In [ ]:
np.random.seed(0)
def shuffle_arrays(*arrays):
    rand_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rand_state)
        np.random.shuffle(array)

shuffle_arrays(features, labels)

In [ ]:
#constructor = lambda last_layer: Dense(num_zone_classes)



training_data = C.input_variable(num_features, dtype = numType)
zone_label = C.input_variable(num_zone_classes, dtype = numType)
crime_label = C.input_variable(num_crime_classes, dtype = numType)


def classifier(output_dim, input_var, num_layers):
    z = Dense(output_dim)(input_var) #FIXME: Change initial variable name
    for i in range(num_layers - 1):
        z = Dense(output_dim)(z)


zone_classifier = classifier(zone_label.shape[0], training_data, num_layers)
crime_classifier = classifier(crime_label.shape[0], training_data, num_layers)

In [8]:
def my_trainer(output_dim, z, loss, eval_error):
    learning_rate = 0.1 #Fixme: Make a global constant so as to be more noticeable?
    
    lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
    learner = C.sgd(z.parameters, lr_schedule)
    loss = C.cross_entropy_with_softmax(z, zone_label)
    eval_error = c.classification_error(z, zone_label)
    trainer = C.Trainer(z, (loss, eval_error), [learner])

(4,)


In [ ]:
zone_trainer = my_trainer(num_zone_classes, zone_classifier)
crime_trainer = my_trainer(num_crime_classes, crime_classifier)

num_samples = len(data)
num_training = num_samples * 2 // 3
num_testing = num_samples - num_training
training_mb_size = 25




In [ ]:
def moving_average(loss_list):
    averages = np.cumsum(np.asarray(loss_list))
    for i in range(len(averages)):
        averages[i] = averages[i] / (i + 1)
    return averages


#######################Modified from Microsoft CNTK101 Tutorial#####################
def print_progress(trainer, mb_num, frequency, display = True):
    loss = "NA"
    if mb_num % frequency == 0:
        loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if display:
            print("Minibatch {0:04n}: Average Loss = {1:.4f}, Average Error = {2:.4f}".format(mb_num, loss, eval_error))
    return mb_num, loss, eval_error

In [ ]:
###########################Modified from Microsoft CNTK101 Tutorial#################
from collections import defaultdict

def training_model(trainer, training_size, mb_size, features, labels, input_var, label_var):
    training_plot_data = defaultdict(list)
    progress_output_freq = 1000
    training_features = features[:training_size]
    training_labels = labels[:training_size]
    
    num_training_mbs = training_size // mb_size
    
    for i in range(num_training_minibatches):
        start_index = i * training_minibatch_size
        end_index = start_index + training_minibatch_size
        feature_batch = training_features[start_index : end_index]
        label_batch = training_labels[start_index : end_index]
        trainer.train_minibatch({input_var : feature_batch, label_var : label_batch})
    
        mb_num, loss, error = print_progress(trainer, i, progress_output_freq)
        if loss != "NA":
            training_plotdata["Batch Number"].append(mb_num)
            training_plotdata["Loss"].append(loss)
            training_plotdata["Error"].append(error)
            
    return training_plotdata

def testing_model(trainer, mb_size, features, labels, input_var, label_var):
    testing_plotdata = defaultdict(list)
    

    for i in range(num_testing_minibatches):
        start_index = i * mb_size
        end_index = start_index + mb_size
        feature_batch = testing_features[start_index : end_index]
        label_batch = testing_labels[start_index : end_index]
    
        eval_error = trainer.test_minibatch({input_var : feature_batch, label_var : label_batch})
    
        if eval_error != "NA":
            testing_plotdata["Batch Number"].append(i)
            testing_plotdata["Error"].append(eval_error)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def

training_plotdata["Average Loss"] = moving_average(training_plotdata["Loss"])
training_plotdata["Average Error"] = moving_average(training_plotdata["Error"])

testing_plotdata["Average Error"] = moving_average(testing_plotdata["Error"])

plt.plot(training_plotdata["Batch Number"], training_plotdata["Average Loss"])
plt.title("Cumulative Average Loss")
plt.xlabel("Minibatch Number")
plt.ylabel("Loss")
plt.show()

print("Average Training Loss = {0:.2f}".format(training_plotdata["Average Loss"][-1]))


plt.hist(testing_plotdata["Loss"])
plt.title("Testing Error")
plt.xlabel("Minibatch Average Error")
plt.ylabel("Frequency")
plt.show()

print("Average Testing Loss = {0:.2f}".format(testing_plotdata["Average Loss"][-1]))

In [ ]:
zone_model_path = "zone_classifier.cmf"
crime_model_path = "crime_classifier.cmf"

zone_classifier.save(zone_model_path)
crime_classifier.save(crime_model_path)

In [ ]:
zone_model = C.load